In [45]:
%cd ../airflow
%pwd

/home/welberthime/Development/fiap/cartolafc/airflow


'/home/welberthime/Development/fiap/cartolafc/airflow'

In [37]:
from include import RawExtractor

extractor = RawExtractor(
    base_url="https://api.github.com/repos/henriquepgomide/caRtola/contents/data",
    path="./data/raw",
)

extractor.extract_static_files("2018")
extractor.extract_dynamic_files("2018")

In [39]:
import pandas as pd
import uuid

In [62]:
clubes_df = pd.read_csv("./data/raw/2018/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2018", "nome.cbf"]].rename(
    columns={"cod.2018": "olderID", "id": "clubeID", "nome.cbf": "nome"}
).drop_duplicates()

partidas_df = pd.read_csv("./data/raw/2018/partidas/1.csv")

partidas_df = partidas_df.merge(
    right=clubes_df,
    left_on="home_team",
    right_on="nome",
    suffixes=(None, "_mandante"),
)

partidas_df = partidas_df.merge(
    right=clubes_df,
    left_on="away_team",
    right_on="nome",
    suffixes=(None, "_visitante"),
)

partidas_df["mandantePlacar"] = partidas_df["score"].str.split(" x ").str[0].astype(int)
partidas_df["visitantePlacar"] = partidas_df["score"].str.split(" x ").str[-1].astype(int)

def get_resultado(row):
    if row["mandantePlacar"] > row["visitantePlacar"]:
        return "Casa"
    elif row["mandantePlacar"] < row["visitantePlacar"]:
        return "Visitante"
    else:
        return "Empate"
    
partidas_df["resultado"] = partidas_df.apply(get_resultado, axis=1)
partidas_df["partidaID"] = [uuid.uuid4().hex for _ in range(len(partidas_df))]
    
partidas_df["temporada"] = 2018
partidas_df = partidas_df.rename(
    columns={
        "clubeID": "clubeMandanteID",
        "clubeID_visitante": "clubeVisitanteID",
        "round": "rodada",
    }
)

partidas_df = partidas_df[[
    "partidaID",
    "clubeMandanteID",
    "clubeVisitanteID",
    "mandantePlacar",
    "visitantePlacar",
    "rodada",
    "resultado",
    "temporada",
]]

partidas_df

,partidaID,clubeMandanteID,clubeVisitanteID,mandantePlacar,visitantePlacar,rodada,resultado,temporada
0,b876958df3c449f0bee11cb82a92298f,283,284,0,1,1,Visitante,2018
1,9b6b1665f410496693c7fc81d17509be,293,284,2,1,21,Casa,2018
2,f6f206c1f65744d0852900b7c52de189,327,284,1,1,30,Empate,2018
3,1e3da772ec874443b54b758ad4a7b63d,287,284,0,0,37,Empate,2018
4,6a0da4a58e9f4f4e84828fdc1fc2753e,267,284,1,0,14,Casa,2018
...,...,...,...,...,...,...,...,...
375,8609468d4532477085b4f77d21acfbe8,262,283,1,0,18,Casa,2018
376,26290882be8040ee931bb657896cbc84,354,283,0,1,9,Visitante,2018
377,e3fced459aa14226ab96f278933db8dd,292,283,0,0,24,Empate,2018
378,703f4bb855c548d697eafa191f9aac21,275,283,3,1,27,Casa,2018


In [63]:
clubes_df = pd.read_csv("./data/raw/2018/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2018", "nome.cartola"]].rename(
    columns={"cod.2018": "olderID", "id": "clubeID", "nome.cartola": "nome"}
).drop_duplicates()

from glob import glob

file_list = glob("./data/raw/2018/rodada/*.csv")
scouts_df = pd.DataFrame()
for file in file_list:
    tmp_df = pd.read_csv(file)
    scouts_df = pd.concat([scouts_df, tmp_df], sort=True)

scouts_df = scouts_df.merge(
    right=clubes_df,
    left_on="atletas.clube.id.full.name",
    right_on="nome",
)

scouts_df = scouts_df.rename(
    columns={
        'atletas.atleta_id': "atletaID",
        'atletas.rodada_id': "rodada",
        'atletas.pontos_num': "pontos",
        'atletas.media_num': "pontosMedia",
        'atletas.preco_num': "preco", 
        'atletas.variacao_num': "precoVariacao",
    },
)

house_by_rounds = partidas_df[["partidaID", "rodada", "clubeMandanteID"]].rename(
    columns={"clubeMandanteID": "clubeID"}
)
visitor_by_rounds = partidas_df[["partidaID", "rodada", "clubeVisitanteID"]].rename(
    columns={"clubeVisitanteID": "clubeID"}
)
club_by_rounds = pd.concat([house_by_rounds, visitor_by_rounds])

scouts_df = scouts_df.merge(
    right=club_by_rounds,
    on=["rodada", "clubeID"],
)

scouts_df = scouts_df[[
    "clubeID",
    "partidaID",
    "atletaID",
    "pontos",
    "pontosMedia",
    "preco",
    "precoVariacao",
    "FS",
    "PE",
    "A",
    "FT",
    "FD",
    "FF",
    "G",
    "I",
    "PP",
    "RB",
    "FC",
    "GC",
    "CA",
    "CV",
    "SG",
    "DD",
    "DP",
    "GS",
]]

scouts_df

,clubeID,partidaID,atletaID,pontos,pontosMedia,preco,precoVariacao,FS,PE,A,...,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
0,284,26f00efb595c4c329e345053a9952be2,98832,0.0,0.00,3.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,284,26f00efb595c4c329e345053a9952be2,99628,0.0,0.00,1.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,284,26f00efb595c4c329e345053a9952be2,99629,0.0,0.00,1.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,284,26f00efb595c4c329e345053a9952be2,100132,0.0,0.00,2.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,284,26f00efb595c4c329e345053a9952be2,100742,0.0,0.00,1.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30950,327,f6f206c1f65744d0852900b7c52de189,99574,0.0,1.20,3.92,0.00,7.0,19.0,1.0,...,NaN,5.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
30951,327,f6f206c1f65744d0852900b7c52de189,97453,0.0,1.48,8.80,0.00,1.0,9.0,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,2.0,10.0,NaN,12.0
30952,327,f6f206c1f65744d0852900b7c52de189,97317,3.5,1.81,2.00,-0.33,5.0,11.0,NaN,...,NaN,17.0,12.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
30953,327,f6f206c1f65744d0852900b7c52de189,97459,0.0,0.00,1.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
posicoes_df = pd.read_csv("./data/raw/2018/posicoes_ids/1.csv")
posicoes_df.rename(
    columns={
        "Cod": "posicaoID",
        "Position": "nome",
        "abbr": "abreviacao",
    },
    inplace=True,
)

posicoes_df = posicoes_df[[
    "posicaoID",
    "nome",
    "abreviacao",
]]

posicoes_df

,posicaoID,nome,abreviacao
0,1,Goleiro,gol
1,2,Lateral,lat
2,3,Zagueiro,zag
3,4,Meia,mei
4,5,Atacante,ata
5,6,Técnico,tec


In [68]:
clubes_df = pd.read_csv("./data/raw/2018/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2018", "nome.cartola"]].rename(
    columns={"cod.2018": "olderID", "id": "clubeID", "nome.cartola": "nome"}
).drop_duplicates()

atletas_df = pd.read_csv("./data/raw/2018/jogadores/1.csv")

atletas_df = atletas_df.merge(
    right=clubes_df,
    left_on="atletas.clube.id.full.name",
    right_on="nome",
)

atletas_df.drop_duplicates(inplace=True)
atletas_df = atletas_df.merge(
    right=posicoes_df,
    left_on="atletas.posicao_id",
    right_on="nome",
)

atletas_df.rename(
    columns={
        "atletas.atleta_id": "atletaID",
        "atletas.apelido": "apelido",
    },
    inplace=True
)

atletas_df = atletas_df[[
    "atletaID",
    "clubeID",
    "posicaoID",
    "apelido",
]]

atletas_df

,atletaID,clubeID,posicaoID,apelido
0,37717,354,1,Fernando Henrique
1,37899,354,1,Renan
2,72294,354,1,Éverson
3,71882,354,1,Diogo Silva
4,71631,275,1,Weverton
...,...,...,...,...
956,81794,265,6,Cláudio Prates
957,84863,263,6,Alberto Valentim
958,37313,263,6,Marcos Paquetá
959,102887,263,6,Bruno Lazaroni


In [69]:
clubes_df = pd.read_csv("./data/raw/2018/times_ids/1.csv")
clubes_df.rename(
    columns={
        "id": "clubeID",
        "nome.cbf": "nome",
    },
    inplace=True,
)

clubes_df = clubes_df[["clubeID", "nome", "abreviacao"]]
clubes_df.drop_duplicates(subset=["clubeID"], keep="last", inplace=True)

clubes_df

,clubeID,nome,abreviacao
0,327,América - MG,AME
1,200,America - RN,OUT
2,373,Atlético - GO,ATL
3,282,Atlético - MG,ATL
6,293,Athlético-PR,ATL
7,314,Avaí - SC,AVA
8,265,Bahia - BA,BAH
9,263,Botafogo - RJ,BOT
10,203,Brasiliense,BRA
13,354,Ceará - CE,CEA
